In [1]:
using BenchmarkTools

# Sparse linear algebra

In [ ]:
"""
    CpBDBt!(C, B, d)

    Compute C += B*D*B'
        where B' is the transpose of B, and D = diag(d1, ..., dn)
        and B is mxn dense, C is mxm (dense), d is nx1 (dense).

    C is modified in-place (no memory allocated),
    only its upper-triangular part is modified.
"""
function CpBDBt!(C::Array{T,2}, B::SparseMatrixCSC{T,Int64}, d::Array{T, 1}) where T
    
    m, k = B.m, B.n
    
    # TODO: remove @assert macro, and throw error instead
    @assert (m, m) == size(C)
    @assert (k,) == size(d)
    
    # compute symmetric rank-k update
    # only the upper-triangular part of C is modified
    temp = zero(T)
    for j=1:m
        for l = 1:k
            @inbounds temp = d[l] * B[j, l]
            if temp == zero(T)
                # skip loop if temp is zero
                continue 
            end
            for i=1:j
                @inbounds C[i, j] = C[i, j] + temp * B[i, l]
            end
        end
    end
    
    return nothing
end

# Data structures

In [ ]:
struct SparseBlockAngular{T}
    m::Int  # number of linking constraints
    n::Int  # number of columns (total)
    R::Int  # number of blocks

    ncols::Array{Int64, 1}  # number of columns in each block [n_1, ..., n_R]
    cols::Array{SparseMatrixCSC{T,Int64}, 1}  # blocks of (sparse) columns [A_1, ..., A_R]
end

In [ ]:
# (Implicit) Cholesky factor of ADA' where A is primal block-angular
mutable struct FactorSparseBlockAngular{T}
    m::Int  # number of linking constraints
    n::Int  # number of columns (total)
    R::Int  # number of blocks
    ncols::Array{Int64, 1}  # number of columns in each block [n_1, ..., n_{R}]
    
    
    d::Array{T, 1}  # Rx1 vector
    eta::SparseMatrixCSC{T,Int64}  # mxR sparse matrix
    
    # Cholesky factor of (dense) Schur complement
    Fc::Base.LinAlg.Cholesky{T,Array{T,2}}
end

# Implicit Cholesky

In [5]:
typeof(sprand(10, 0.1))

SparseVector{Float64,Int64}

In [ ]:
"""
    cholSparseBlockAngular(A, theta)
"""
function cholSparseBlockAngular(A::SparseBlockAngular{T}, theta::Array{T, 1}) where T
    
    # Schur complement
    C = zeros(T, A.m, A.m)
    
    # diagonal
    d = zeros(T, A.R)
    eta = [spzeros(T, A.m) for r=1:R]  # columns of eta
    
    colidx = 1
    for r=1:A.R
        theta_ = theta[colidx:(colidx+A.ncols[r]-1)]
        colidx += A.ncols[r]
        
        # diagonal elements
        d[r] = oneunit(T) / sum(theta_)
        
        # lower factors
        eta[r] = A.cols[r] * theta_
        
        # Schur complement
        # TODO
#         C += A.cols[r] * spdiagm(theta_) * A.cols[r]'
#         C += (-d[r]) * (eta[r] * eta[r]')
    end
    
    # coalesce eta
    eta = hcat(eta...)
    
    # compute Cholesky factor of C
    Fc = cholfact(Symmetric(C))
    
    # create implicit Cholesky factor
    F = FactorSparseBlockAngular(A.m, A.n, A.R, A.ncols, d, eta, Fc)
    
    return F
    
end